In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
from sklearn.tree import plot_tree
from sklearn.linear_model import LogisticRegression


data = pd.read_csv('waste management.csv')
data.head()

print(data.shape)
data.isnull().sum()

FileNotFoundError: [Errno 2] No such file or directory: 'waste management.csv'

In [ ]:
# Model deveplopment - Logistics Regression
x=data[['waste_type','material_composition','recycling_potential','toxicity_level']]
y=data['management_strategy']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
model=LogisticRegression()
model.fit(x_train,y_train)

y_pred=model.predict(x_test)
print(y_pred)

# accuracy
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy=",accuracy)

In [ ]:
# Model deveplopment - Decision Tree
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
model=DecisionTreeClassifier(max_depth=3,random_state=42)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print(y_pred)

# accuracy
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy : ",accuracy)

In [ ]:
# Model deveplopment - Random Forest
rf_model  = RandomForestClassifier(n_estimators=100,random_state=42)
rf_model.fit(x_train,y_train)
print(y_pred)

# accuracy
y_pred_rf = rf_model.predict(x_test)
accuracy_score(y_test,y_pred_rf)

In [ ]:
# SVC
from sklearn.svm import SVC
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

svm_model = SVC(kernel='linear')   # you can also try 'rbf'
svm_model.fit(x_train_scaled, y_train)

y_pred = svm_model.predict(x_test_scaled)
print(y_pred)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy =", accuracy)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

model = SVC(kernel='rbf',C=1.0,gamma='scale')
model.fit(x_train,y_train)



In [ ]:
import joblib

joblib.dump(rf_model, "random_forest_model.pkl")
joblib.dump(svm_model, "svm_model.pkl")

In [ ]:
import gradio as gr
import joblib
model = joblib.load('svm_model.pkl')

In [ ]:
def predict_managment_strategy(waste_type,material_composition,recycling_potential,toxicity_level):
  input_data=pd.DataFrame([{
      'waste_stype':waste_type,
      'material_composition':material_composition,
      'recycling_potential':recycling_potential,
      'toxicity_level':toxicity_level
  }])

  prediction=model.predict(input_data)[0]

  if prediction == 0:
    return 'Non-Recyclable'
  else:
    return 'Recyclable'

In [ ]:
waste_type_input = gr.Slider(minimum=1, maximum=3, step=1, label="Waste Type(1-3)")
material_composition_input = gr.Slider(minimum=0.0, maximum=1.0, step=0.01, label="Material Composition(0-1)")
recycling_potential_input = gr.Slider(minimum=0.0, maximum=1.0, step=0.01, label="Recycling Potential(0-1)")
toxicity_level_input = gr.Slider(minimum=0.0, maximum=100.0, step=0.1, label="Toxicity Level(0-100)")

Interface = gr.Interface(
    fn=predict_managment_strategy,
    inputs=[waste_type_input,material_composition_input,recycling_potential_input,toxicity_level_input],
    outputs="text",
    title="Waste Management Strategy Predictor",
    description="Enter waste characteristics to predict the managment strategy(Non-Recyclable or Recyclable)"
)

Interface.launch()